In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import tensorflow as tf
import numpy as np
import json

In [3]:
from eventcnn.datasources.davis import DavisDataset
from eventcnn.datasources.eventsource import EventSource

In [4]:
# Taken mostly from 
# https://indico.io/blog/tensorflow-data-inputs-part1-placeholders-protobufs-queues/
def read_and_decode_single_example(filename, metadata):
    filename_queue = tf.train.string_input_producer([filename], 
                                                    num_epochs=None)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={
            'label': tf.FixedLenFeature([3], tf.float32),
            'events': tf.FixedLenFeature(
                [metadata["cols"], 
                 metadata["rows"], 
                 metadata["event_layers"],
                 metadata["channels"]], tf.int64)
        })
    label = features['label']
    events = features['events']
    return label, events

In [16]:
metadata = json.load(open("records/metadata.json"))
label, events = read_and_decode_single_example("records/training.tfrecords",
                                               metadata)

label_batch, events_batch = tf.train.shuffle_batch(
    [label, events], batch_size=3,
    capacity=9,
    min_after_dequeue=6)

In [17]:
sess = tf.Session()

init = tf.initialize_all_variables()
sess.run(init)
tf.train.start_queue_runners(sess=sess)

[<Thread(Thread-8, started daemon 123145366233088)>,
 <Thread(Thread-9, started daemon 123145371488256)>]

In [18]:
l1, e1 = sess.run([label_batch, events_batch])

In [19]:
l1

array([[ -2.13636784e-04,  -3.99345881e-05,   2.04206823e-04],
       [ -1.61422984e-04,   9.65436193e-05,   7.14321095e-06],
       [ -1.84619275e-04,  -1.09350593e-04,   1.32923815e-04]], dtype=float32)

In [20]:
e1.shape

(3, 120, 90, 4, 2)